In [65]:
import pandas as pd
import os
from netCDF4 import Dataset

In [66]:
iso3 = pd.read_csv("ciso3.txt", sep=" ")

In [67]:
iso3 = iso3.set_index('iso3')

In [68]:
results = [i for i in os.listdir('FSC_Scenarios') if 'Results' in i]

In [69]:
results

['NAF_High_Results.csv',
 'MBBF_High_Results.csv',
 'MBBF_Low_Results.csv',
 'CPA_Low_Results.csv',
 'NAF_Mid_Results.csv',
 'FSU_High_Results.csv',
 'FSU_Mid_Results.csv',
 'EUR_Low_Results.csv',
 'CPA_High_Results.csv',
 'FSU_Low_Results.csv',
 'EUR_Mid_Results.csv',
 'NAF_Low_Results.csv',
 'CPA_Mid_Results.csv',
 'MBBF_Mid_Results.csv',
 'EUR_High_Results.csv']

In [70]:
first = True
for r in results:
    region = r.split("_")[0]
    shock = r.split("_")[1]
    if first:
        df = pd.read_csv(f"FSC_Scenarios/{r}")
        df["Region"] = region
        df["Shock"] = shock
        first = False
    else:
        df_ = pd.read_csv(f"FSC_Scenarios/{r}")
        df_["Region"] = region
        df_["Shock"] = shock
        df = df.append(df_)

In [71]:
df['iso3'] = df['Unnamed: 0']

In [72]:
df = df.join(iso3, how="left", on="iso3")
df["Country"] = df["name"]

In [73]:
df.loc[df['iso3']=='SSD', 'Country'] = 'South Sudan'

In [74]:
del(df["Unnamed: 0"])
del(df["FAO"])
del(df["name"])

In [75]:
df

,P0,R0,dR,dC,S0,Region,Shock,iso3,Country
0,1.072782e+12,2.471600e+11,-1.426044e+09,0.000000e+00,7119771960000,NAF,High,ARM,Armenia
1,8.288590e+13,1.568130e+13,-1.463493e+09,0.000000e+00,83905213988000,NAF,High,AUS,Australia
2,3.093241e+14,5.591427e+13,-1.006519e+11,0.000000e+00,333674785966000,NAF,High,IND,India
3,0.000000e+00,5.494300e+12,-4.302259e+06,0.000000e+00,4033194000,NAF,High,IDN,Indonesia
4,3.661526e+13,2.946882e+13,0.000000e+00,0.000000e+00,36615277116000,NAF,High,IRN,Iran (Islamic Republic of)
5,1.201986e+13,2.553764e+12,-1.853192e+11,0.000000e+00,14378872196000,NAF,High,IRQ,Iraq
6,2.214086e+12,1.779377e+11,-1.219426e+06,0.000000e+00,2214596370000,NAF,High,IRL,Ireland
7,5.888333e+11,9.098160e+11,0.000000e+00,0.000000e+00,588860622000,NAF,High,ISR,Israel
8,2.507710e+13,1.998370e+12,-6.548114e+07,0.000000e+00,25089479154000,NAF,High,ITA,Italy
9,0.000000e+00,2.478280e+11,-3.345232e+10,0.000000e+00,-22110132206000,NAF,High,CIV,Cote d'Ivoire


In [ ]:
df.to_csv("FSC.csv", index=False)

## TWIST Ingestion

In [55]:
ds = Dataset("TWIST-Scenarios/EUR_high_shock/output.nc", "r")

In [62]:
print(ds.variables.keys())

dict_keys(['time', 'years', 'months', 'P_nom', 'P_def', 'P_sim', 'P_ref', 'P_nat', 'P_floor', 'P_thres', 'P_ceil', 'I_int', 'QE_n', 'QE_p', 'h_dist', 'h_dist_ref', 'WM_supplies', 'FD', 'FDref', 'prod_storage', 'cons_storage', 'I_max_cons', 'max_stu', 'stocks_dat', 'prod_names'])


In [64]:
ds['P_def']

<class 'netCDF4._netCDF4.Variable'>
float32 P_def(time)
    _FillValue: -100.0
unlimited dimensions: 
current shape = (90,)
filling on